In [ ]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4


Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 15.4 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.5/819.5 kB 10.5 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 8.3 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.1/26.1 MB 70.5 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 34.6 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.9 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 10.0 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-6rx11ewa/overlay/local/bin
    changing mode of /tmp/pip-build-env-6rx11ewa/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-6rx11ewa/overlay/

In [ ]:
import re
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [ ]:

model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

In [ ]:

model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

llama-2-13b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/9.76G [00:00<?, ?B/s]

In [ ]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )


AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [ ]:

from google.colab import files


pd= files.upload()


Saving tagged_data_llama_256_1.csv to tagged_data_llama_256_1.csv


In [ ]:
import pandas as pd
import random
df = pd.read_csv('tagged_data_llama_256_1.csv')


In [ ]:
df = df[['reviews', 'Correct review ', 'Incorrect review ']]
df.head()

,reviews,Correct review,Incorrect review
0,Not bad but could be better,1,0
1,Taking biotin 5000mcg from October first week ...,1,0
2,My nails grew in noticeably thicker and strong...,1,0
3,I took biotin for 1 week. I started having mu...,1,0
4,I used to have very thin nails that bent when ...,1,0


In [ ]:
df_incorrect = df.loc[df['Incorrect review '] == 1]['reviews'].reset_index(drop=True)
df_correct = df.loc[df['Correct review '] == 1]['reviews'].reset_index(drop=True)
df_correct[random.choice(range(len(df_correct)))]

'i am not sure if melatonin is giving me  hand tremors. yhey seem to be worse after i started taking i.'

In [ ]:
df_incorrect[random.choice(range(len(df_incorrect)))]

'just started to take vitamen d  hoping for help with neuropothy'

In [ ]:
#Step 6: Generating the Response

In [ ]:
for _ in range(20):
    prompt = f'''Examples of drug review ratings:
    1. {df_correct[random.choice(range(len(df_correct)))][:256]}
     ANSWER: 1
    2. {df_incorrect[random.choice(range(len(df_incorrect)))][:256]}
     ANSWER: 0
    3. {df_correct[random.choice(range(len(df_correct)))][:256]}
     ANSWER: 1
    4. {df_incorrect[random.choice(range(len(df_incorrect)))][:256]}
     ANSWER: 0
    5. {df_correct[random.choice(range(len(df_correct)))][:256]}
     ANSWER: 1

   You are a product review evaluator. Your task is to assess the correctness of only following review for dietary supplements and provide a label (1 for correct, 0 for incorrect):
    "{df['reviews'][random.choice(range(len(df)))]}"
    '''
    prompt_template=f'''SYSTEM:You are a helpful, respectful, and honest assistant. Your task is to accurately evaluate drug reviews for correctness based on the provided examples.

    USER: {prompt}

    ASSISTANT:
    '''
    response=lcpp_llm(prompt=prompt_template, max_tokens=512, temperature=0.5, top_p=0.95,
                      repeat_penalty=1.2, top_k=150,
                      echo=True)
    print(response)
    print(response["choices"][0]["text"])
    response_text = response["choices"][0]["text"]
    matches = re.findall(r'ASSISTANT:.*?([01])', response_text, re.DOTALL)

    print(matches)


{'id': 'cmpl-3007d87e-a689-481b-a8a7-74e4bc144061', 'object': 'text_completion', 'created': 1701620257, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM:You are a helpful, respectful, and honest assistant. Your task is to accurately evaluate drug reviews for correctness based on the provided examples.\n\n    USER: Examples of drug review ratings:\n    1. OMG!  GAS!!  DIARRHEA!!  BLOATING!!  NAUSEA!!  Even two days later!!  I hope the benefits outweigh this misery!\n     ANSWER: 1\n    2. I think my doctor is having me take to much, I have been so tired. Is this possible. He gave the vit d from his office. none of the optional information applies to  me\n     ANSWER: 0\n    3. Take gingo for the pain in my legs, it works. I think it’s important to chosse a good lab that you can relly on. I use Pharma Nord and on the lab site the have a complet 

Llama.generate: prefix-match hit


{'id': 'cmpl-629434eb-4d18-40ee-b860-8270e893bdd5', 'object': 'text_completion', 'created': 1701620277, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM:You are a helpful, respectful, and honest assistant. Your task is to accurately evaluate drug reviews for correctness based on the provided examples.\n\n    USER: Examples of drug review ratings:\n    1. I have only taken mega red krill for 2 days. It has caused my stomach to be upset and caused a lot of gas and groweling. I am stopping until I talk with my Dr. tomorrow.\n     ANSWER: 1\n    2. Ginko iks supposed to help with dizziness,but side effects can cause diziness,confusing.\n     ANSWER: 0\n    3. I am taking this drug as a preventive measure while trying to conceive.  I have had no side effects and find it very easy to take.  Though I won\'t be certain how well it works until I become

Llama.generate: prefix-match hit


{'id': 'cmpl-91769ac7-6f8a-4649-bbc2-13ca02e3c033', 'object': 'text_completion', 'created': 1701620315, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM:You are a helpful, respectful, and honest assistant. Your task is to accurately evaluate drug reviews for correctness based on the provided examples.\n\n    USER: Examples of drug review ratings:\n    1. I take 50,000 units once per week for low vitamin D readings on my last blood check up. I am having headaches that seem to be coming more frequently and the pain is getting worse. This is the end of my second week of a 6 week treatment.\n     ANSWER: 1\n    2. web md is afraid to show any positive results about cs they are like all the rest.  its all about the money!  i wonder why they dont have a homemade option under the brand product used?\n     ANSWER: 0\n    3. I started taking Mega Red k

...

{'id': 'cmpl-b5ee6e31-b019-44fb-bb30-7e511271434c', 'object': 'text_completion', 'created': 1701620805, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM:You are a helpful, respectful, and honest assistant. Your task is to accurately evaluate drug reviews for correctness based on the provided examples.\n\n    USER: Examples of drug review ratings:\n    1. My only complaint, it takes several months to see results. Now my hair is coming in like crazy!!! So easy!!\n     ANSWER: 1\n    2. The article starts out saying that there are no known uses for silver, but then in one small sentence afterwards says How does it work?    "Colloidal silver can kill certain germs by binding to and destroying proteins."    Big Pharma hard at work...\n     ANSWER: 0\n    3. bone pain is not experinced but fingers can not be bend\n     ANSWER: 1\n    4. I was lookin

Llama.generate: prefix-match hit


{'id': 'cmpl-9db31e65-0ca9-491b-9beb-08ae3b86e1ee', 'object': 'text_completion', 'created': 1701620846, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM:You are a helpful, respectful, and honest assistant. Your task is to accurately evaluate drug reviews for correctness based on the provided examples.\n\n    USER: Examples of drug review ratings:\n    1. I was having no luck with Ambien or Trazadone anymore, so decided to try Melantonin - 3 mg.  It worked the first night and for the past week.  Just hope it continues.  What a blessing!\n     ANSWER: 1\n    2. I havent been on it long enough to really contribute..I would like to know,Iam using it for hair thinning,nerves, how long does it take and are there side effects???\n     ANSWER: 0\n    3. Unbelievable results with no side effects!!\n     ANSWER: 1\n    4. DEAR SIR,LAS MONTH I HAD CHACKE

Llama.generate: prefix-match hit


{'id': 'cmpl-221f460d-54d8-4ca1-af07-0a65214de399', 'object': 'text_completion', 'created': 1701620863, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM:You are a helpful, respectful, and honest assistant. Your task is to accurately evaluate drug reviews for correctness based on the provided examples.\n\n    USER: Examples of drug review ratings:\n    1. Loosing hair, thought I would give a try. Hair stopped falling out.\n     ANSWER: 1\n    2. Prescribed by my Dr....I am taking 50,000 mg weekly....1 weekly\n     ANSWER: 0\n    3. Careful! Some of us are allergic to Vitamin D3 because it is made from lanolin from sheep\'s wool and we are allergic to sheep\'s wool because of the lanolin. Apparently, the allergen is still in the Vitamin D3.\n     ANSWER: 1\n    4. Satisfied\n     ANSWER: 0\n    5. So my heart dr Sid that folgrad reduce my plaque

In [ ]:
!pip install scikit-learn

true_labels = [1,1,1,1,1,1,1,1,1,1]
predicted_labels = [1,1,1,1,0,0,1,0,0,1]

correct_predictions = sum(1 for true, predicted in zip(true_labels, predicted_labels) if true == predicted)

accuracy = correct_predictions / len(true_labels)

print(f"Точность модели: {accuracy * 100:.2f}%")

Точность модели: 60.00%
